In [1]:
import archetypal as ar
import pandas as pd
import os
from path import Path

# to disable, %matplotlib inline
#%matplotlib notebook
%matplotlib inline

ar.config(use_cache=True)

c:\users\mariana\desktop\projects\buildingtechnology\containerfarmep\archetypal_files\archetypal\archetypal\utils.py:117: UserWarning: The TRNSYS path does not exist. Please set the TRNSYS path with the --trnsys-default-folder option
  warnings.warn(


In [2]:
# ----------------------------------------------------------
# ---------------- GLOBAL VARIABLES ------------------------
# ----------------------------------------------------------
ep_version = '9.2'
climate_region = '5a'

all_buildings = ['LargeOffice',
                 'MediumOffice',
                 'SmallOffice',
                 'Warehouse',
                 'Stand-aloneRetail',
                 'StripMall',
                 'PrimarySchool',
                 'SecondarySchool',
                 'Supermarket',
                 'QuickService Restaurant',
                 'FullService Restaurant',
                 'Hospital',
                 'Outpatient Health Care',
                 'SmallHotel',
                 'LargeHotel',
                 'MidriseApartment']
all_epochs = ["Pre1980", "Post1980", "New2004"]
energy_header = ['Cooling (kWh/m2)',
                  'Heating (kWh/m2)',
                  'Lights (kWh/m2)',
                  'Equipment (kWh/m2)',
                  'DHW (kWh/m2)',
                  'EUI (kWh/m2)']
sim_header = ['EP','UMI','error']

date_index = pd.date_range(start='2018/01/01', end='2019/1/1', closed='left', freq='H')

In [3]:
def getResultsDF(idf):
    #conditioned floor area to normalize
    area = idf.net_conditioned_building_area #m2
    date_index_local = pd.date_range(start='2018/01/01', end='2019/1/1', closed='left', freq='H')
    
    #start create datagrame
    results = pd.DataFrame(index=date_index_local)

    # calculate ratio of cooling and heating energy
    cooling_energy = (
        "CoolingCoils__EnergyTransfer"
    )
    heating_energy = (
        "HeatingCoils__EnergyTransfer",
        "Baseboard__EnergyTransfer"
    )
    
    total_cooling_energy = pd.Series(0,index=date_index_local)
    for item in cooling_energy:
        try:
            total_cooling_energy += (idf.meters.OutputMeter[item].values("kWh"))
        except KeyError:
            pass  # pass if meter does not exist for model
    
    total_heating_energy = pd.Series(0,index=date_index_local)
    for item in heating_energy:
        try:
            total_heating_energy += (idf.meters.OutputMeter[item].values("kWh"))
        except KeyError:
            pass  # pass if meter does not exist for model
        
    ratio_cooling = total_cooling_energy / (total_cooling_energy+total_heating_energy)
    ratio_heating = total_heating_energy / (total_cooling_energy+total_heating_energy)

      
    #calcualte fans electricity for cooling and heating

    if 'Fans__Electricity' in idf.meters.OutputMeter.__dict__:
        fans_cooling = idf.meters.OutputMeter.Fans__Electricity.values('kWh') * ratio_cooling /area
        fans_heating = idf.meters.OutputMeter.Fans__Electricity.values('kWh') * ratio_heating /area       
    else:
        fans_cooling = pd.Series(0,index=date_index_local)
        fans_heating = pd.Series(0,index=date_index_local)
    
   
    #calculate pumps electricity for cooling and heating
    
    if 'Pumps__Electricity' in idf.meters.OutputMeter.__dict__:
        pumps_cooling = idf.meters.OutputMeter.Pumps__Electricity.values('kWh') * ratio_cooling /area
        pumps_heating = idf.meters.OutputMeter.Pumps__Electricity.values('kWh') * ratio_heating /area
    else:
        pumps_cooling = pd.Series(0,index=date_index_local)
        pumps_heating = pd.Series(0,index=date_index_local)
    
    #cooling
    if 'Cooling__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] = idf.meters.OutputMeter.Cooling__Electricity.values(units = ('kWh'))/area
    else:
        results['Cooling (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'HeatRejection__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += idf.meters.OutputMeter.HeatRejection__Electricity.values(units = ('kWh'))/area

    if 'Cooling__Gas' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += idf.meters.OutputMeter.Cooling__Gas.values(units = ('kWh'))/area
  
    if 'Cooling__DistrictCooling' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += idf.meters.OutputMeter.Cooling__DistrictCooling.values(units = ('kWh'))/area

    if 'Fans__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += fans_cooling
        
    if 'Pumps__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += pumps_cooling

    
    #heating
    if 'Heating__Gas' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] = idf.meters.OutputMeter.Heating__Gas.values(units = ('kWh'))/area 
    else:
        results['Heating (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'Heating__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] += idf.meters.OutputMeter.Heating__Electricity.values(units = ('kWh'))/area
        
    if 'Heating__DistrictHeating' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] += idf.meters.OutputMeter.Heating__DistrictHeating.values(units = ('kWh'))/area
        
    if 'Heating__Oil' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] += idf.meters.OutputMeter.Heating__Oil.values(units = ('kWh'))/area 
        
    if 'Fans__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += fans_heating
        
    if 'Pumps__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += pumps_heating

    
    #lights
    if 'InteriorLights__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Lights (kWh/m2)'] = idf.meters.OutputMeter.InteriorLights__Electricity.values(units = ('kWh'))/area
    else:
        results['Lights (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    
    #equipment
    if 'InteriorEquipment__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Equipment (kWh/m2)'] = idf.meters.OutputMeter.InteriorEquipment__Electricity.values(units = ('kWh'))/area
    else:
        results['Equipment (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'Refrigeration__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Equipment (kWh/m2)'] += idf.meters.OutputMeter.Refrigeration__Electricity.values(units = ('kWh'))/area
        
    
    #hot water 
    if 'WaterSystems__Gas' in idf.meters.OutputMeter.__dict__:
        results['DHW (kWh/m2)'] = idf.meters.OutputMeter.WaterSystems__Gas.values(units = ('kWh'))/area
    else:
        results['DHW (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'WaterSystems__Electricity' in idf.meters.OutputMeter.__dict__:
        results['DHW (kWh/m2)'] += idf.meters.OutputMeter.WaterSystems__Electricity.values(units = ('kWh'))/area

    
    #total EUI
    results['EUI (kWh/m2)'] = results.apply(lambda row: 
                                              row['DHW (kWh/m2)']+
                                              row['Equipment (kWh/m2)']+
                                              row['Lights (kWh/m2)']+
                                              row['Heating (kWh/m2)']+
                                              row['Cooling (kWh/m2)'], 
                                              axis = 1)
    
    return results

In [4]:
# get weather file
ep_weather = next(iter(Path('WeatherFile').files("*.epw")), None)
if not ep_weather:
    raise Exception("No weather file found")

In [10]:
all_buildings_test = ['SmallOffice',
                      'Warehouse']

loop through IDFs (test: Warehouse)

In [11]:
# create blank dictionary to store IDFs
idfs = {}

simulate=True

#start excel document to save results
with pd.ExcelWriter(f'{climate_region}_ResultsComparison.xlsx') as writer:

    #loop through all building types
    # replace with all_buildings
    for bldg in all_buildings: 

        #empty list of results dataframes for the epoch
        epoch_dfs = []

        # loop through all epochs
        for epoch in all_epochs:
            ep_file = next(iter(Path("IDF").files(f"*{bldg}{epoch}*.idf")), None)
            if not ep_file:
                raise FileNotFoundError(f"Cound not find file for {bldg}{epoch}")

            #read idf
            idfs[f"{bldg}{epoch}"] = ar.IDF(ep_file, ep_weather, as_version = ep_version)

            if simulate:
                #simulate idf
                idfs[f"{bldg}{epoch}"].simulate()

            #get results and make dataframe for epoch + building combo
            results = getResultsDF(idfs[f"{bldg}{epoch}"])

            #group results by monthly and att totals row
            monthly_results = results.groupby(results.index.month).sum()
            monthly_results.index.map(str)

            # add total row
            monthly_tot = monthly_results.append(monthly_results.sum().rename('Annual'))
            
            #####################################################
            # read in umi file ###### SAM TO ADD CODE HERE ######
            #####################################################
            # this will need to be replaced with a function that reads 
            # results from pyumi for each building-epoch combo
            py_umi = pd.DataFrame(0,index=date_index,columns=energy_header)  
            #####################################################
            
            #reformat umi results
            umi_month = py_umi.groupby(py_umi.index.month).sum()
            umi_month.index.map(str)
            umi = umi_month.append(umi_month.sum().rename('Annual'))

            # create error dataframe
            error_df = (umi-monthly_tot)/monthly_tot

            #concat results with umi and error dataframes
            results_all = pd.concat([monthly_tot, umi, error_df], 
                                         keys = sim_header, 
                                         names=['Simulation Type','Energy Type'], 
                                         axis = 1)
            #swap the order of the headers and sort
            results_swapped = results_all.swaplevel(axis=1).sort_index(axis=1).loc[:, (energy_header, sim_header)]

            # add floor area row
            area_blg = idfs[f"{bldg}{epoch}"].net_conditioned_building_area
            area_df = pd.DataFrame(area_blg,index=['Conditioned Area m2'],columns=results_swapped.columns)
            results_with_area = pd.concat([results_swapped,area_df])

            #add temporary results to list of epoch results
            epoch_dfs.append(results_with_area)

        # concat all dfs
        annual = pd.concat(epoch_dfs, keys=all_epochs, names = ['Epoch','Energy Type','Simulation Type'], axis = 1)
        annual.index.name = 'Month'
    
        # write results of building to new sheet
        annual.to_excel(writer, sheet_name=bldg)